# Using LaVague for QA Automation

In this notebook, we'll show how LaVague can be used to automatically generate pytest files from a Gherkin test definition

We will use LaVague to autonomously run the test and record xpath and actions. We'll then use an LLM to generate assert statements and the final reusable test file. 

# Test cases
**There are currently two examples test cases, only run the cell you want to test**

## Test case 1: Amazon Cart
- Accept cookies
- Perform a search
- Click on a product
- Add it to cart
- Access the cart page
- Remove the item from cart
- Verify that the cart is empty

In [1]:
URL = "https://www.amazon.fr/"

GHERKIN = """Feature: Amazon Cart

  Scenario: Add and remove a single product from cart
    Given I am on the homepage
    When I click "Accepter" to accept cookies
    And I enter "Harry Potter et la Chambre des Secrets" into the search bar and press Enter
    And I click on "Harry Potter et la Chambre des Secrets" in the search results
    And I am on the product details page
    And I click on the "Ajouter au panier" button
    And I am taken to a confirmation page
    And I click on "Aller au panier"
    And I am taken to the cart page
    And I remove the product from the cart by clicking on "Supprimer"
    Then the cart should be empty
"""

## Test case 2: LaPoste Shipping Calculator
- Accept cookies
- Navigate to the shipping calculator
- Select values for package size and weight
- Verify the price is as expected

In [2]:
URL = "https://www.laposte.fr/"

GHERKIN = """Feature: Shipping cost calculator

  Scenario: Estimate shipping costs for a large package
    Given I am on the homepage
    When I click on "J'accepte" to accept cookies
    And I click on "Envoyer un colis"
    And I click on the "Format du colis" dropdown under "Dimension"
    And I click on "Volumineux & tube" from the dropdown results
    And I enter 15 in the "Poids" field
    And I wait for the cost to update
    Then the cost should be "34,70 €"
"""

# Running a test case with LaVague

## Parsing the Gherkin
We parse the Gherkin file to extract the assert statement, this will be usefull when we want to generate the assert code

In [11]:
FEATURE_FILE_NAME = "demo.feature"
TEST_FILE_NAME = "demo_test.py"
scenarios = GHERKIN.split("Scenario:")
parsed_scenarios = []
for scenario in scenarios[1:]:
    scenario_name, *scenario_steps = scenario.strip().split("\n")
    parsed_scenarios.append(
        {
            "name": scenario_name.strip(),
            "steps": [step.strip() for step in scenario_steps],
        }
    )

def prepare_scenario(scenario):
    return "\n".join(scenario["steps"]), scenario["steps"][-1]


steps, assert_statement = prepare_scenario(parsed_scenarios[0])

print(assert_statement)


Then the cost should be "34,70 €"


## Init LaVague

We create a standard LaVague agent and open the URL

In [4]:
from lavague.core import  WorldModel, ActionEngine
from lavague.core.agents import WebAgent
from lavague.drivers.selenium import SeleniumDriver
from lavague.core.retrievers import SemanticRetriever
from selenium.webdriver.chrome.options import Options

from llama_index.llms.openai import OpenAI
from llama_index.multi_modal_llms.openai import OpenAIMultiModal
from llama_index.legacy.readers.file.base import SimpleDirectoryReader

selenium_driver = SeleniumDriver(headless=False)
world_model = WorldModel()
action_engine = ActionEngine(selenium_driver)
agent = WebAgent(world_model, action_engine)

agent.get(URL)

2024-07-29 16:54:58,668 - INFO - Screenshot folder cleared


## Start the agent
We start the agent and record the steps taken as a `pandas` dataframe

In [5]:
agent.run(f"Run this test case step by step: {steps}", log_to_db=True)

logs = agent.logger.return_pandas()

2024-07-29 16:55:04,686 - INFO - Thoughts:
- The current screenshot shows the homepage of La Poste.
- The first step in the test case is to click on "J'accepte" to accept cookies.
- The "J'accepte" button is likely to be present on the homepage, possibly in a cookie consent banner.
- The next step should involve locating and clicking on the "J'accepte" button to proceed.

Next engine: Navigation Engine
Instruction: Click on the "J'accepte" button to accept cookies.
2024-07-29 16:55:17,133 - INFO - Thoughts:
- The current screenshot shows the homepage of La Poste.
- The objective is to follow a series of steps to calculate the cost of sending a package.
- The first step, accepting cookies, has been completed.
- The next step is to click on "Envoyer un colis" to proceed with the package sending process.

Next engine: Navigation Engine
Instruction: Click on "Envoyer un colis".
2024-07-29 16:55:30,069 - INFO - Thoughts:
- The current screenshot shows the page for sending a package ("Envoye

Connected to SQLite
Table created or altered successfully
Log insert complete


# Processing a test case

## Extract run steps and code
After the agent has finished running, we extract instructions and actions taken from the logs. 
We also get the screenshot of the last page visited. 

In [6]:
# we will remove chain of thought comments to only keep the actions
def remove_comments(code):
    return '\n'.join([line for line in code.split('\n') if not line.strip().startswith('#')])

# get all actions
actions = "\n".join(logs["code"].dropna())

# clean data
logs['action'] = logs['code'].dropna().apply(remove_comments)
cleaned_logs = logs[['instruction', 'action']].fillna('')
actions = '\n\n'.join(cleaned_logs['instruction'] + ' ' + cleaned_logs['action'])

# get last page screenshot
last_page_screenshot = SimpleDirectoryReader(logs.iloc[-1]["screenshots_path"]).load_data() # load last screenshot taken

print(actions)

Click on the "J'accepte" button to accept cookies. 
- actions:
    - action:
        args:
            xpath: "/html/body/div/div[2]/div[2]/button"
        name: "click"

Click on "Envoyer un colis". 
- actions:
    - action:
        args:
            xpath: "/html/body/div/div/div/div[2]/div/main/div/div[2]/div[4]/div/div/a[3]"
            value: ""
        name: "click"

Click on the "Format du colis" dropdown under "Dimension". 
- actions:
    - action:
        args:
            xpath: "/html/body/div/div/div/div/main/div/div[2]/div[2]/div/div/div/div/div/div/div/div[3]/div[2]/fieldset/div"
            value: ""
        name: "click"

Click on "Volumineux & tube" from the dropdown results. 
- actions:
    - action:
        args:
            xpath: "/html/body/div/div/div/div/main/div/div[2]/div[2]/div/div/div/div/div/div/div/div[3]/div[2]/fieldset/div[2]/div/label[2]"
            value: ""
        name: "click"

Enter "15" in the "Poids" field. 
- actions:
    - action:
        args

## Retrieve nodes for assert
Using the `SemanticRetriever`, we fetch nodes that could be relevant to generate the assert statement

In [7]:
retriever = SemanticRetriever(embedding=action_engine.python_engine.embedding, xpathed_only=True)
html = selenium_driver.get_html()
nodes = retriever.retrieve(f"{assert_statement}" , html.splitlines())

In [8]:
# you can run this cell to display all nodes returned by the retriever
from IPython.display import HTML

for e in nodes: 
    display(HTML(e))

# Generate the `pytest` file
Use recorded data about the site to generate a pytest-bdd file

We use a multi modal LLM (`gpt-4o`) to generate the final code file

In [9]:
gpt4o = OpenAIMultiModal("gpt-4o")
gpt4o.max_new_tokens = 2000 # 300 by default, we increase it to make sure our pytest file doesn't get trucated

# we'll clean the triple quoted answers from the LLM
def clean_output(markdown_code_block):
    return markdown_code_block.replace("```python", "").replace("```", "").replace("```\n", "")

## Building the prompt

We use a prompt that combines general instructions, examples and the recorded run data to generate the pytest file

In [12]:
SYSTEM_PROMPT = f"""
You are an expert in software testing frameworks and Python code generation. You answer in python markdown only and nothing else, don't include anything after the last backticks. Your task is to:
1. Process a test case, existing actions (with their associated xpath) that was ran during the test by an agent, relevant HTML nodes, and the screenshot of the last state of the page.
2. Generate an assert statement for the last condition of the test case. THis is the most important, you need to generate one assert statement based on the provided `Then` step. 
3. Package everything in a pytest-bdd file following best practices. 
Requirements:
- Use descriptive function names and name the scenario appropriately.
- Use execute_script to handle potential ElementClickInterceptedException during clicks.
- Always use time.sleep(3) if waiting is required
- Include fixtures, scenario, Gherkin-style definitions, etc.
- Use try-except blocks to catch exceptions and raise pytest.fail for assert condition steps as needed.
- Do not generate asserts for 'Given', 'When', or 'And' steps; only generate asserts for 'Then' steps.
- Always use provided selenium code that was already executed to find valid selectors for the final pytestfile. 
- You answer in python code only and nothing else.
"""

EXAMPLES = """import pytest
from pytest_bdd import scenarios, given, when, then, parsers
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import ElementClickInterceptedException
import time
import random

# Constants
BASE_URL = 'https://example.com'

# Scenarios
scenarios('example.feature')

# Fixtures
@pytest.fixture
def browser():
   driver = webdriver.Chrome()
   driver.implicitly_wait(10)
   yield driver
   driver.quit()

# Steps
@given('I am on the example website')
def go_to_homepage(browser):
   browser.get(BASE_URL)
   WebDriverWait(browser, 10).until(
       EC.presence_of_element_located((By.TAG_NAME, "body"))
   )

@when(parsers.parse('I click the "{button_text}" button'))
def click_button(browser, button_text):
   button = WebDriverWait(browser, 10).until(
       EC.element_to_be_clickable((By.XPATH, f"//button[contains(text(), '{button_text}')]"))
   )
   try:
       browser.execute_script("arguments[0].click();", button)
   except ElementClickInterceptedException:
       pytest.fail(f"Failed to click the '{button_text}' button")

@when(parsers.parse('I enter "{text}" in the "{field_name}" field'))
def enter_text(browser, text, field_name):
   input_field = WebDriverWait(browser, 10).until(
       EC.presence_of_element_located((By.XPATH, f"//input[@placeholder='{field_name}' or @name='{field_name}']"))
   )
   input_field.clear()
   input_field.send_keys(text)

@when('I select a random option from the dropdown')
def select_random_option(browser):
   dropdown = WebDriverWait(browser, 10).until(
       EC.presence_of_element_located((By.TAG_NAME, "select"))
   )
   options = dropdown.find_elements(By.TAG_NAME, "option")
   random_option = random.choice(options)
   random_option.click()

@when('I wait for the page to load')
def wait_for_page_load(browser):
   time.sleep(5)
   WebDriverWait(browser, 10).until(
       EC.presence_of_element_located((By.TAG_NAME, "body"))
   )
   time.sleep(2)  # Additional wait to ensure page is fully loaded

@then(parsers.parse('I should see the text "{expected_text}" on the page'))
def verify_text_presence(browser, expected_text):
   try:
       WebDriverWait(browser, 10).until(
           EC.presence_of_element_located((By.XPATH, f"//*[contains(text(), '{expected_text}')]"))
       )
   except Exception as e:
       pytest.fail(f"Failed to find the text '{expected_text}' on the page: {str(e)}")

@then(parsers.parse('the "{element_name}" should be visible'))
def verify_element_visibility(browser, element_name):
   try:
       element = WebDriverWait(browser, 10).until(
           EC.visibility_of_element_located((By.XPATH, f"//*[@id='{element_name}' or @name='{element_name}' or contains(@class, '{element_name}')]"))
       )
       assert element.is_displayed(), f"The element '{element_name}' is not visible"
   except Exception as e:
       pytest.fail(f"Failed to verify visibility of '{element_name}': {str(e)}")

@then(parsers.parse('the page title should be "{expected_title}"'))
def verify_page_title(browser, expected_title):
   try:
       time.sleep(3)
       WebDriverWait(browser, 10).until(EC.title_is(expected_title))
   except Exception as e:
       pytest.fail(f"Page title does not match '{expected_title}': {str(e)}")
"""



PROMPT = f"""{SYSTEM_PROMPT}

Generate a valid pytest-bdd file with the following inputs and examples to guide you:
Feature file name: {FEATURE_FILE_NAME}\n
URL: {URL}\n
Full feature to be tested: \n{GHERKIN}\n
Already executed (instruction + actions):\n{actions}\n
Assert statement (Then) to test: {assert_statement}
Chunk of HTML that may be usefull to generate the assert statement. Generate a general statement that checks for the assert using these nodes: {nodes}\n
Examples:\n\n{EXAMPLES}
"""

## Generate the test
Call the LLM with the final prompt then clean the ouput

In [13]:
generated_pytest = gpt4o.complete(PROMPT, image_documents=last_page_screenshot).text
generated_pytest = clean_output(generated_pytest)

In [14]:
print(generated_pytest)


import pytest
from pytest_bdd import scenarios, given, when, then, parsers
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import ElementClickInterceptedException
import time

# Constants
BASE_URL = 'https://www.laposte.fr/'

# Scenarios
scenarios('demo.feature')

# Fixtures
@pytest.fixture
def browser():
    driver = webdriver.Chrome()
    driver.implicitly_wait(10)
    yield driver
    driver.quit()

# Steps
@given('I am on the homepage')
def go_to_homepage(browser):
    browser.get(BASE_URL)
    WebDriverWait(browser, 10).until(
        EC.presence_of_element_located((By.TAG_NAME, "body"))
    )

@when('I click on "J\'accepte" to accept cookies')
def click_accept_cookies(browser):
    button = WebDriverWait(browser, 10).until(
        EC.element_to_be_clickable((By.XPATH, "/html/body/div/div[2]/div[2]/but

## Write test to file

We'll write two generated files to disk: 
- `.feature` contains the test scenarios written in the Gherkin syntax
- `.py` the actual automated test that we'll run with pytest

In [15]:
import os
with open(FEATURE_FILE_NAME, "w") as file:
        file.write(GHERKIN)
        
with open(TEST_FILE_NAME, "w") as file:
        print("WRITING FILE")
        file.write(generated_pytest)

WRITING FILE


# Run tests

We can finally run our generated test to see it in action

In [16]:
!pytest demo_test.py --full-trace

============================= test session starts ==============================
platform darwin -- Python 3.10.14, pytest-8.2.1, pluggy-1.5.0
rootdir: /Users/palmi/work/repos/fix-token-counter-name-lookup/LaVague
configfile: pyproject.toml
plugins: anyio-4.3.0, bdd-7.1.2
collected 1 item                                                               

demo_test.py .                                                           [100%]

============================== 1 passed in 8.29s ===============================
